<a href="https://colab.research.google.com/github/hirogami/2020sCmpEngQnn/blob/main/Copy_of_Copy_of_Fine_Tune_DistilGPT2_and_Generate_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This the copy of the tutorial:
https://colab.research.google.com/github/tripathiaakash/DistilGPT2-Tutorial/blob/main/distilgpt2_fine_tuning.ipynb

I have come across some errors and corrected the code. The hashtag symbol (#) refers to the parts of the code that I have modified.


Here is a tutorial about generating text using a SOTA inspired language generation model, distilgpt2. This model lighter in weight and faster in language generation than the original OpenAI GPT2. Using this tutorial, you can train a language generation model which can generate text for any subject in English. Here, we will generate movie reviews by fine-tuning distilgpt2 on a sample of IMDB movie reviews.


In [ ]:
# download the IMDB sample dataset
!wget http://files.fast.ai/data/examples/imdb_sample.tgz

--2023-02-06 00:39:45--  http://files.fast.ai/data/examples/imdb_sample.tgz
Resolving files.fast.ai (files.fast.ai)... 104.26.3.19, 172.67.69.159, 104.26.2.19, ...
Connecting to files.fast.ai (files.fast.ai)|104.26.3.19|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.fast.ai/data/examples/imdb_sample.tgz [following]
--2023-02-06 00:39:45--  https://files.fast.ai/data/examples/imdb_sample.tgz
Connecting to files.fast.ai (files.fast.ai)|104.26.3.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 571827 (558K) [application/x-gtar-compressed]
Saving to: ‘imdb_sample.tgz’

imdb_sample.tgz     100%[===================>] 558.42K  1.52MB/s    in 0.4s    

2023-02-06 00:39:45 (1.52 MB/s) - ‘imdb_sample.tgz’ saved [571827/571827]



In [ ]:
### Extract the csv file from the uploaded tgz file
import tarfile
with tarfile.open('imdb_sample.tgz', 'r:gz') as tar:
    tar.extractall()

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('imdb_sample/texts.csv')

In [ ]:
### This is how the CSV look like
data

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False
...,...,...,...
995,negative,There are many different versions of this one ...,True
996,positive,Once upon a time Hollywood produced live-actio...,True
997,negative,Wenders was great with Million $ Hotel.I don't...,True
998,negative,Although a film with Bruce Willis is always wo...,True


For Finetuning distilgpt2, we just need the text field

In [ ]:
texts = list(set(data['text']))

Store the reviews in a txt file where each line of txt file is a single review 

In [ ]:
file_name = 'testing.txt'
with open(file_name, 'w') as f:
    f.write(" |EndOfText|\n".join(texts))

Now, let's come to Transformers by Huggingface, and unleash the Transformers (Autobots... just kidding)

In [ ]:
# install the latest version of transformers with pip
!pip install git+https://github.com/huggingface/transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-kryr697v
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-kryr697v
  Resolved https://github.com/huggingface/transformers to commit 59d5edef34ae0fa56065a2e863736d4f133c558b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 74.7 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.27.0.dev0-py3-none-any.whl size=6434401 sha256=921939d69e681bcebebde7359067d8e1d8827c9aefb03b4904a6bd2f1d0b58c6
  Stored in directory: /tmp/pip-ephem-wheel-cache-ok9k3t42/wheels/42/68/45/c63edff61c292f2dfd4df4ef6522dcbecc603e7af82

In [ ]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 124287, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 124287 (delta 14), reused 18 (delta 10), pack-reused 124260
Receiving objects: 100% (124287/124287), 119.95 MiB | 30.46 MiB/s, done.
Resolving deltas: 100% (93258/93258), done.


Make 2 directories. 

1) weights - for storing the weights of distilgpt2

2) tokenizer - for storing the tokenizer of distilgpt2

In [ ]:
# create the directories with mkdir in the terminal
!mkdir weights tokenizer

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Now, its time for Training (or fine tuning) distilgpt2 with IMDB reviews
Given below is a command containing few parameters to help Transformers finetune distilgpt2. now, let's understand what these parameters mean

1) output_dir: It is the weights_dir we made where our finetuned model will be stored in the form of checkpoints

2) model_name_or_path: It tells the kind of model we are currently dealing with

3) per_device_train_batch_size: It tells the batch size for each gpu

4) do_train: It tells pytorch to start training mode

5) train_file: This is where we give the input text data 

6) num_train_epochs: Number of epochs for finetuning


Now, let the training begin...

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 3.2 MB/s eta 0:00:00


In [ ]:
weights_dir = "output"

In [ ]:
cmd = '''
python transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path distilgpt2 \
    --train_file {0} \
    --do_train \
    --num_train_epochs 3 \
    --overwrite_output_dir \
    --per_device_train_batch_size 2 \
    --output_dir {1}
'''.format(file_name, weights_dir)

In [ ]:
!{cmd}

INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
igno

Although, Huggingface provides a run_generation.py file for language generation. Running it from a command (as it takes the input), makes it load the model and the tokenizer everytime you run the file which slows downs generation. To reduce the I/O overhead, I have restructured the run_generation.py file in the following code which only loads the model and tokenizer once in a model and a tokenizer object and we can use these objects to generate text over and over again

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def get_model_tokenizer(weights_dir, device = 'cuda'):
    print("Loading Model ...")
    model = GPT2LMHeadModel.from_pretrained(weights_dir)
    model.to('cuda')
    print("Model Loaded ...")
    tokenizer = GPT2Tokenizer.from_pretrained(weights_dir)
    return model, tokenizer

def generate_messages(
    model,
    tokenizer,
    prompt_text,
    stop_token,
    length,
    num_return_sequences,
    temperature = 0.7,
    k=20,
    p=0.9,
    repetition_penalty = 1.0,
    device = 'cuda'
):

    MAX_LENGTH = int(10000)
    def adjust_length_to_model(length, max_sequence_length):
        if length < 0 and max_sequence_length > 0:
            length = max_sequence_length
        elif 0 < max_sequence_length < length:
            length = max_sequence_length  # No generation bigger than model size
        elif length < 0:
            length = MAX_LENGTH  # avoid infinite loop
        return length
        
    length = adjust_length_to_model(length=length, max_sequence_length=model.config.max_position_embeddings)

    encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")

    encoded_prompt = encoded_prompt.to(device)

    output_sequences = model.generate(
            input_ids=encoded_prompt,
            max_length=length + len(encoded_prompt[0]),
            temperature=temperature,
            top_k=k,
            top_p=p,
            repetition_penalty=repetition_penalty,
            do_sample=True,
            num_return_sequences=num_return_sequences,
        )

    if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()

    generated_sequences = []

    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
        #print("=== GENERATED SEQUENCE {} ===".format(generated_sequence_idx + 1))
        generated_sequence = generated_sequence.tolist()

        # Decode text
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

        # Remove all text after the stop token
        text = text[: text.find(stop_token) if stop_token else None]

        # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
        total_sequence = (
            prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
        )

        generated_sequences.append(total_sequence)
    return generated_sequences


In [ ]:

model, tokenizer = get_model_tokenizer(weights_dir, device = 'cuda')


Loading Model ...
Model Loaded ...


In [ ]:
temperature = 1.0
k=400
p=0.9
repetition_penalty = 1.0
num_return_sequences = 5
length = 1000
stop_token = '|EndOfText|'
prompt_text = "The main casts are"

In [ ]:
%%time
generate_messages(
    model,
    tokenizer,
    prompt_text,
    stop_token,
    length,
    num_return_sequences,
    temperature = temperature,
    k=k,
    p=p,
    repetition_penalty = repetition_penalty
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CPU times: user 12.4 s, sys: 631 ms, total: 13 s
Wall time: 12.9 s


["The main casts are Charlie Cooper (Hanks), Jake Gyllenhaal (Taj), John Smith (Logan), Robert Leguizamo (Richard Kean), James Daly (Marianne Cox), Philip McManus (Robert Paxton), Juman Gordon (Benny Dukler), Kurt Coleman (Robert Morris), Jason Hocken (Nancy Bitt); Carl Hendry (Franklin Selips), Nicholas Hoxton (Daniel Day-Lewis), and William James Smith Jr. (Brian Newman) when they enter a town together, and escape from the town's government. When a young girl meets the handsome woman on a camping trip, John tells her about a romantic encounter he's had with him for the past three years, when she tries to convince him of the friendship he has. He tells him that he had a romantic relationship with the old man before, when he had been married and gave him the keys to the town hall. John then turns the conversation around to the girl to remind him of his brother's role in the story of the original TV series, the novels, or the characters, and tells him about his friends' relationship, in